**Author:** A.S. Grm (aleksander.grm@fpp.uni-lj.si)

**Date:** 2024

<hr>

# Sextant Height Corrections

Measured heights obtained from a sextant, referred to as **sextant height** $H_s$, need to undergo several corrections to derive the **observed height** $H_o$ used for positioning. These corrections are performed in sequential steps:

1. **Index Correction**:
   $$H = H_s + I + D$$
   - $I$: index error (device error)
   - $D$: dip error (error due to horizon dip)
   - $H$: **apparent** or topocentric height

2. **Refraction Correction**:
   $$H_r = H - \rho$$
   - $\rho$: refraction error

3. **Parallax Correction**:
   $$H_q = H_r + \text{PA}$$
   - $\text{PA}$: parallax error

4. **Semi-diameter Correction**:
   $$H_o = H_q \pm \text{Sd}$$
   - $\text{Sd}$: semi-diameter error
   - $H_o$: observer or **true geocentric** height
  
For the refraction it is possible to use **Smart** or **Bennet** atmosphere model!

In [ ]:
import os, sys

# add custom modules and astro data path 
pp = '../nav_tools/'
sys.path.append(pp)

In [ ]:
# needed libraries
import math as mat
import navtools as nt

ref_model = ['smart', 'bennet']

In [ ]:
# Dip horizon error
# - h: observer height above sea level in [m]

def dip_err(h):

    return -1.76 * mat.sqrt(h)/60

In [ ]:
# Refraction error

def refraction_err(H, model):

    if model == ref_model[0]:
        if H < 15:
            err = (34.133 + 4.197*H + 0.00428*H**2)/(1 + 0.505*H + 0.0845*H**2)
        else:
            err = 0.97127/mat.tan(H*mat.pi/180) - 0.00137/mat.tan(H*mat.pi/180)**3
    else:
        y1 = H + 7.31/(H+4.4)
        err1 = 1/mat.tan(y1*mat.pi/180)

        y2 = 14.7*err1 + 13
        err2 = -0.06 * mat.sin(y2*mat.pi/180)

        err = err1 + err2
        
    return err

In [ ]:
# Meteo refraction error
# - p in [hPa]; reference p=1010 hPa 
# - T in [C] reference T=10 C

def meteo_ref_err(T,p):

    k = 0.28 * p / (T + 273)

    return k

In [ ]:
# Parallax error

def parallax_err(HP, Hr):

    return HP * mat.cos(Hr*mat.pi/180)

In [ ]:
# Main program

# *** inputs ***
Hs = [20,12.6] # [deg,min] # sextant height
I = -2         # Index error in minutes
h = 20         # Observer height above sea level in meters
sd = 0         # Body semi-diameter: + lower limb, - upper limb
HP = 0         # Zero altitude paralax
T = 30         # Outer temperature in celsius
p = 1010       # Outer pressure in hPa

# ---------------
ie = I/60
de = dip_err(h)
print(' dip: {:}'.format(nt.prettyPrintAlt(de)))
H = nt.dms2dd(Hs) + ie + de
print(' H: {:}'.format(nt.prettyPrintAlt(H)))
print()

re1 = refraction_err(H, ref_model[0]); print(' re1: {:}'.format(nt.prettyPrintAlt(re1)))
re2 = refraction_err(H, ref_model[1]); print(' re2: {:}'.format(nt.prettyPrintAlt(re2))) 
me = meteo_ref_err(T,p); print(' meteo k: {:.5f}'.format(me))
re1m = me*re1
re2m = me*re2

Hr = H - re1
Hrm = H - re1m
#Hr = H - re2
#Hrm = H - re2m
print('       Hr: {:}'.format(nt.prettyPrintAlt(Hr)))
print(' meteo Hr: {:}'.format(nt.prettyPrintAlt(Hrm)))
print()

pe = parallax_err(HP, Hr)
Hq = Hr + pe
print(' Hq: {:}'.format(nt.prettyPrintAlt(Hq)))

Ho = Hq + sd/60
print(' Ho: {:}'.format(nt.prettyPrintAlt(Ho)))